In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
model = tf.keras.applications.VGG16(weights="imagenet")
img_path = "C:\\Users\\1\\Desktop\\reinforcment_deep_ML\\cats_and_dogs_dataset\\train\\dogs\\202.jpg"

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))

x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.vgg16.preprocess_input(x)

predictions = model.predict(x)
print("predicted: ", tf.keras.applications.vgg16.decode_predictions(predictions, top=3)[0])
print(np.argmax(predictions[0]))

1/1 [==============================] - 0s 251ms/step
predicted:  [('n02110185', 'Siberian_husky', 0.42510137), ('n03218198', 'dogsled', 0.29216036), ('n02109961', 'Eskimo_dog', 0.237662)]
250


In [8]:
siberian_husky_output = model.output[:, 250]
last_conv_layer = model.get_layer("block5_conv3")

grads = tf.keras.backend.gradients(siberian_husky_output, last_conv_layer.output)[0]
pooled_grads = tf.keras.backend.mean(grads, axis=(0, 1, 2))
iterate = tf.keras.backend.function([model.input],
                                    [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, acis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.imshow(heatmap)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.